# Example Query for sensitive patients

Takes list of mutated genes and forms it into a mock patient profile. Queries out system in the form of:
P(survival_time > 1000 | g1_mut = True, g2_mut = True, ..., gn_mut = True). 
Returned is a knowledge graph contained probability of survival time and a description containing sensitive patients used in the inference.

In [1]:
import requests

In [2]:
# Function: buildQuery
#
# Input:
# -----------
# list of gene tuples containing gene name and ENSEMBL Identifiers
#
# Output
# -----------
# A knowledge graph with decorated edges for 
# P(survival_time > 1000 | RAF1 = Mutated, BRAF = Mutated)
# and description containing sensitive patients dump

def buildQuery(genes):
    # empty response
    response = { "query_graph": dict(),
                 "knowledge_graph": dict(),
                 "response": dict()
               }

    # empty query graph
    response["query_graph"] = { "edges": [],
                                "nodes": []
                              }

    # empty knowledge graph
    response["knowledge_graph"] = { "edges": [],
                                    "nodes": []
                                  }

    # empty response graph
    response["results"] = { "node_bindings": [],
                            "edge_bindings": []
                          }

    # nodes
    nodeCount = 0
    # edges
    edgeCount = 0

    # add in evidence genes
    
    for g in genes:
        response['query_graph']['nodes'].append({ 'id':'n{}'.format(nodeCount),
                                                  'type':'Gene',
                                                  'name':'{}'.format(g[0]),
                                                  'curie':'{}'.format(g[1])
                                               })
        nodeCount += 1

    # grouping for genes
    response['query_graph']['nodes'].append({ 'id':'n{}'.format(nodeCount),
                                              'type':'gene_grouping'
                                           })
    nodeCount += 1
    
    # link genes over grouping
    for n in response['query_graph']['nodes'][:-1]:
        response['query_graph']['edges'].append({ 'id':'e{}'.format(edgeCount),
                                                  'type':'part_of',
                                                  'curie':['SEMMEDDB:PART_OF'],
                                                  'source_id':n['id'],
                                                  'target_id':'n{}'.format(nodeCount-1)
                                               })
        edgeCount += 1

    # patient node
    response['query_graph']['nodes'].append({ 'id':'n{}'.format(nodeCount),
                                              'type':'patient',
                                              'curie':['UMLSSC:T101']
                                           })
    nodeCount += 1

    # link gene group to patient
    response['query_graph']['edges'].append({ 'id':'e{}'.format(edgeCount),
                                              'type':'expressed_in',
                                              'curie':['RO:0002206'],
                                              'source_id':'n{}'.format(nodeCount-2),
                                              'target_id':'n{}'.format(nodeCount-1)
                                           })
    edgeCount += 1

    # survival node
    response['query_graph']['nodes'].append({ 'id': 'n{}'.format(nodeCount),
                                              'type': 'PhenotypicFeature',
                                              'curie': 'CHPDART:SURVIVAL',
                                              'operator': '>=',
                                              'value': '1000'
                                           })
    nodeCount += 1

    # link patient to survival
    response['query_graph']['edges'].append({ 'id':'e{}'.format(edgeCount),
                                              'type':'has_phenotype',
                                              'source_id':'n{}'.format(nodeCount-2),
                                              'target_id':'n{}'.format(nodeCount-1)
                                           })
    edgeCount += 1

    # BKB target
    response['probability_targets'] = [('Survival_Time', '>=', 1000)]

    return response

In [3]:
def readGenes():
    f = open('genes.txt', 'r')
    gene_list = f.read()
    gene_list = gene_list.split(',')
    return gene_list

In [1]:
# list of genes we can query over
#gene_list = readGenes()
#print(gene_list)

genes = [('RAF1','[ENSEMBL:ENSG00000132155.12]'),('BRAF','[ENSEMBL:ENSG00000157764.13]')]

response = buildQuery(genes)
payload = {'query': response, 'reasoner_id': 'unsecret'}
r = requests.post('http://chp.thayer.dartmouth.edu/submitQuery/', json=payload)
